In [1]:
import numpy as np
import os
import torch
from sklearn.externals import joblib

from pytorch_utils.models import SparseModel, SparseModelEmbed

In [2]:
data_path = 'data/'
features_path = os.path.join(data_path, 'features', str(0))
label_path = os.path.join(data_path, 'labels')

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
features_dict = joblib.load(os.path.join(features_path, 'features.pkl'))
label_dict = joblib.load(os.path.join(label_path, 'label_dict.pkl'))

In [5]:
outcome = 'mortality'

In [7]:
data_dict = {split: features_dict[split]['features'] for split in features_dict.keys()}
outcome_dict = {split : label_dict[split][outcome] for split in label_dict.keys()}

In [9]:
config_dict = {
    'input_dim' : data_dict['train'].shape[1],
    'output_dim' : 2,
    'lr' : 1e-3,
    'num_epochs' : 10,
    'batch_size' : 256,
    'iters_per_epoch' : None,
}

In [11]:
%%time
model = SparseModelEmbed(config_dict)
result = model.train(data_dict, outcome_dict)
model.predict(data_dict, outcome_dict, keys = ['test'])

Epoch 0/9
----------
Phase: train:
 loss: 0.102829,
 auc: 0.787447, auprc: 0.118535, brier: 0.022629,
Phase: val:
 loss: 0.086132,
 auc: 0.854036, auprc: 0.172330, brier: 0.020122,
Best model updated
Epoch 1/9
----------
Phase: train:
 loss: 0.059329,
 auc: 0.952740, auprc: 0.510458, brier: 0.015182,
Phase: val:
 loss: 0.084897,
 auc: 0.854276, auprc: 0.195811, brier: 0.019587,
Best model updated
Epoch 2/9
----------
Phase: train:
 loss: 0.045733,
 auc: 0.977293, auprc: 0.697536, brier: 0.011792,
Phase: val:
 loss: 0.086246,
 auc: 0.852490, auprc: 0.208619, brier: 0.019448,
Epoch 3/9
----------
Phase: train:
 loss: 0.037860,
 auc: 0.986316, auprc: 0.792125, brier: 0.009725,
Phase: val:
 loss: 0.089222,
 auc: 0.844728, auprc: 0.210896, brier: 0.019420,
Epoch 4/9
----------
Phase: train:
 loss: 0.032475,
 auc: 0.990709, auprc: 0.848453, brier: 0.008272,
Phase: val:
 loss: 0.093072,
 auc: 0.836015, auprc: 0.211233, brier: 0.019446,
Epoch 5/9
----------
Phase: train:
 loss: 0.028445,
 auc:

In [ ]:
%%time
model = SparseModel(config_dict)
result = model.train(data_dict, outcome_dict)
model.predict(data_dict, outcome_dict, keys = ['test'])

Epoch 0/9
----------
Phase: train:
 loss: 0.369275,
 auc: 0.629570, auprc: 0.044339, brier: 0.045391,
Phase: val:
 loss: 0.268438,
 auc: 0.707389, auprc: 0.077343, brier: 0.030677,
Best model updated
Epoch 1/9
----------
Phase: train:
 loss: 0.226232,
 auc: 0.770120, auprc: 0.109066, brier: 0.030892,
Phase: val:
 loss: 0.243193,
 auc: 0.736017, auprc: 0.092827, brier: 0.029046,
Best model updated
Epoch 2/9
----------
Phase: train:
 loss: 0.171132,
 auc: 0.826287, auprc: 0.179783, brier: 0.026613,
Phase: val:
 loss: 0.227673,
 auc: 0.744643, auprc: 0.098835, brier: 0.027645,
Best model updated
Epoch 3/9
----------
Phase: train:
 loss: 0.132861,
 auc: 0.868071, auprc: 0.269610, brier: 0.022876,
Phase: val:
 loss: 0.216875,
 auc: 0.751403, auprc: 0.102936, brier: 0.027535,
Best model updated
Epoch 4/9
----------
Phase: train:
 loss: 0.105675,
 auc: 0.899181, auprc: 0.364645, brier: 0.019816,
Phase: val:
 loss: 0.208983,
 auc: 0.756874, auprc: 0.107001, brier: 0.027634,
Best model updated
